0. Pré-requisitos

In [8]:
!pip install wikipedia
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)
root_path = "/content/drive/My Drive/Colab Notebooks/data/img"
if not os.path.exists(root_path):
  os.mkdir(root_path)

Mounted at /content/drive


1. Recorra ao package do wikipedia [https://pypi.org/project/wikipedia/] para criar um dataset de 2000 imagens relacionadas com duas temáticas distintas à sua escolha (e.g., covid e desporto).

In [15]:
import random
import wikipedia
import requests
import concurrent.futures

max_workers = 4
base_url = "https://pt.wikipedia.org/w/api.php"

def download_image(image):
  try:
    image_title = image["title"]
    PARAMS = {
      "action": "query",
      "format": "json",
      "prop": "imageinfo",
      "titles": image_title,
      "iiprop": "url"
    }
    response = requests.get(url=base_url, params=PARAMS)
    image_data = response.json()
    page_id = next(iter(image_data["query"]["pages"]))
    image_info = image_data["query"]["pages"][page_id].get("imageinfo", [])
    if not image_info:
      return False
    image_url = image_info[0]["url"]
    filename = image_url.split("/")[-1]
    #filename = filename[:-4] + "_" + str(random.randint(0, 9999)) + filename[-4:] # Uncomment if you want unique names for the images
    if filename in os.listdir(root_path):
      return False
    response = requests.get(image_url)
    if response.status_code < 200 or response.status_code > 299:
      return False
    with open(os.path.join(root_path, filename), "wb") as image_file: # Open a new file in binary mode
      image_file.write(response.content)
      return True
  except:
    return False

def extract_page_images(url):
  S = requests.Session()
  PARAMS = {
    "action": "query",
    "format": "json",
    "prop": "images",
    "titles": url.split("/")[-1]
  }
  response = S.get(url=base_url, params=PARAMS)
  data = response.json()
  page_id = next(iter(data["query"]["pages"]))
  images = data["query"]["pages"][page_id].get("images", [])
  images_downloaded = 0
  with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_images = {executor.submit(download_image, image): image for image in images}
    for future in concurrent.futures.as_completed(future_images):
      if future.result():
        images_downloaded +=1
  return images_downloaded


wikipedia.set_lang("pt")  # Define o idioma dos artigos para português
themes = [
  'esoterismo',
  'alquimia',
  'angeologia',
  'astrologia',
  'bruxaria',
  'budismo',
  'confucionismo',
  'cristianismo',
  'cabala',
  'candomblé',
  'deus',
  'deuses',
  'eneagrama',
  'feitiçaria',
  'hinduísmo',
  'islamismo',
  'judaísmo',
  'kabbalah',
  'magia',
  'numerologia',
  'ocultismo',
  'religião',
  'religiões',
  'umbanda',
  'xintoísmo',
]
articles_found = 0
images_downloaded = 0
visited_urls = []
for theme in themes:
  print(f"Buscando imagens em artigos com o tema: {theme}")
  results = wikipedia.search(theme, results=1000)  # Ajuste o número de resultados conforme necessário
  for result in results:
    try:
      page = wikipedia.page(result)
      if page.url in visited_urls:
        print(f"{page.url} já foi visitada, pulando.")
        continue
      epoch_downloads = extract_page_images(page.url)
      visited_urls.append(page.url)
      images_downloaded += epoch_downloads
      articles_found += 1
      if epoch_downloads:
        print(f"{epoch_downloads} novas imagens extraídas do artigo: {page.url}")
      if len(os.listdir(root_path)) > 2000:
        print("Limite de 2000 imagens ultrapassado. Interrompendo execução.")
        break
    except wikipedia.exceptions.DisambiguationError as e:
      print(f"Erro de desambiguação para {result}, pulando.")
    except wikipedia.exceptions.PageError as e:
      print(f"Página não encontrada para {result}, pulando.")
    except:
      print(f"Erro na requisição para {page.url}, pulando.")
  print()
print(f"Total de artigos encontrados: {articles_found}")
print(f"Total de novas imagens extraidas: {images_downloaded}")

Buscando imagens em artigos com o tema: esoterismo
1 novas imagens extraídas do artigo: https://pt.wikipedia.org/wiki/Deus
1 novas imagens extraídas do artigo: https://pt.wikipedia.org/wiki/Sobrenatural


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Erro de desambiguação para Arcano, pulando.
Erro de desambiguação para Bioenergia, pulando.
Erro na requisição para Religião comparada, pulando.
1 novas imagens extraídas do artigo: https://pt.wikipedia.org/wiki/Mitologia_guarani
1 novas imagens extraídas do artigo: https://pt.wikipedia.org/wiki/Albert_Pike
Lista de países por principais grupos religiosos já foi visitada, pulando.
O Livro de Thoth (Aleister Crowley) já foi visitada, pulando.
Era de Capricórnio já foi visitada, pulando.
1 novas imagens extraídas do artigo: https://pt.wikipedia.org/wiki/Tanakh
Erro de desambiguação para Will.i.am, pulando.
Não-teísmo já foi visitada, pulando.
1 novas imagens extraídas do artigo: https://pt.wikipedia.org/wiki/Dogme_et_Rituel_de_la_Haute_Magie
Madame du Barry (1919) já foi visitada, pulando.
Paradoxo da omnipotência já foi visitada, pulando.

Buscando imagens em artigos com o tema: alquimia
Alquimia já foi visitada, pulando.
Alquimia (desambiguação) já foi visitada, pulando.
Tábua de esmer

In [16]:
len(os.listdir(root_path))

2002